In [7]:
%run ../common_notebooks/setup_spark_connection.ipynb
from delta.connect.tables import DeltaTable

In [9]:
df = spark.createDataFrame([(1, "Starter"), (2, "Pro")], ["id", "plan"])
df.write.format("delta").mode("overwrite").save("/opt/spark/delta-tables/table2")
print("Delta table created successfully!")

Delta table created successfully!


In [8]:
# 1. Read the Delta table we created
print("=== Reading Delta Table ===")
delta_df = spark.read.format("delta").load("/opt/spark/delta-tables/table2")
delta_df.show()

=== Reading Delta Table ===
+---+----------+
| id|      plan|
+---+----------+
|  3|Enterprise|
|  1|   Starter|
|  4|     Basic|
|  2|       Pro|
+---+----------+



In [10]:
# 2. Check table schema and metadata
print("\n=== Table Schema ===")
delta_df.printSchema()
print("\n=== Table Count ===")
print(f"Number of records: {delta_df.count()}")


=== Table Schema ===
root
 |-- id: long (nullable = true)
 |-- plan: string (nullable = true)


=== Table Count ===
Number of records: 2


In [11]:
# 4. Add more data (Append mode)
print("\n=== Appending New Data ===")
new_data = spark.createDataFrame([(3, "Enterprise"), (4, "Basic")], ["id", "plan"])
new_data.write.format("delta").mode("append").save("/opt/spark/delta-tables/table2")

# Read updated table
updated_df = spark.read.format("delta").load("/opt/spark/delta-tables/table2")
print("Updated table:")
updated_df.show()


=== Appending New Data ===
Updated table:
+---+----------+
| id|      plan|
+---+----------+
|  3|Enterprise|
|  1|   Starter|
|  4|     Basic|
|  2|       Pro|
+---+----------+



In [12]:
deltaTable = DeltaTable.forPath(spark, "/opt/spark/delta-tables/table2")

# 5. Update existing records
print("\n=== Updating Records ===")
deltaTable.update(
    condition="id = 1",
    set={"plan": "'Starter Pro'"}
)

# Read after update
print("After update:")
spark.read.format("delta").load("/opt/spark/delta-tables/table2").show()


=== Updating Records ===


InvalidPlanInput: No handler found for extension

JVM stacktrace:
org.apache.spark.sql.connect.common.InvalidPlanInput
	at org.apache.spark.sql.connect.planner.SparkConnectPlanner.$anonfun$transformRelationPlugin$3(SparkConnectPlanner.scala:253)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.sql.connect.planner.SparkConnectPlanner.transformRelationPlugin(SparkConnectPlanner.scala:253)
	at org.apache.spark.sql.connect.planner.SparkConnectPlanner.$anonfun$transformRelation$1(SparkConnectPlanner.scala:235)
	at org.apache.spark.sql.connect.service.SessionHolder.$anonfun$usePlanCache$3(SessionHolder.scala:477)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.sql.connect.service.SessionHolder.usePlanCache(SessionHolder.scala:476)
	at org.apache.spark.sql.connect.planner.SparkConnectPlanner.transformRelation(SparkConnectPlanner.scala:147)
	at org.apache.spark.sql.connect.execution.SparkConnectPlanExecution.handlePlan(SparkConnectPlanExecution.scala:74)
	at 

In [ ]:
# 6. Delete records
print("\n=== Deleting Records ===")
deltaTable.delete("id = 4")

# Read after delete
print("After delete:")
spark.read.format("delta").load("/opt/spark/delta-tables/table2").show()

In [ ]:
# 7. Upsert (Merge) operation
print("\n=== Upsert/Merge Operation ===")
merge_data = spark.createDataFrame([(2, "Pro Max"), (5, "Premium")], ["id", "plan"])

deltaTable.alias("target").merge(
    merge_data.alias("source"),
    "target.id = source.id"
).whenMatchedUpdate(set={"plan": "source.plan"}).whenNotMatchedInsert(values={"id": "source.id", "plan": "source.plan"}).execute()

print("After merge:")
spark.read.format("delta").load("/opt/spark/delta-tables/table2").show()

# 8. Time travel queries
print("\n=== Time Travel Queries ===")
# Read table at version 0
print("Version 0:")
spark.read.format("delta").option("versionAsOf", 0).load("/opt/spark/delta-tables/table2").show()

# 9. Vacuum old files (cleanup)
print("\n=== Vacuum Operation ===")
# Note: Vacuum removes files older than retention period
deltaTable.vacuum(0)  # 0 hours for demo (use 168 hours/7 days in production)

print("\n=== Final Table State ===")
spark.read.format("delta").load("/opt/spark/delta-tables/table2").show()
deltaTable.history().select("version", "timestamp", "operation").show()